In [3]:
import qiskit as q
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, IBMQ, execute

# Configuramos las credenciales de IBMQ
IBMQ.save_account('151c59e6f71d7a5c2788c4c846aa50a37456463c6cb37c8ef8d6ef006552e8136b86bcaffb196a892bbd7d9176c1239869df90a56891ef40cc4b849819001e6b',
                  hub='ibm-q-education', group='uni-comp-madrid-1', project='architecure-prog', overwrite=True)
provider = IBMQ.load_account()


def qft(qubits):
    circuit = QuantumCircuit(qubits)
    for i in range(len(qubits)):
        circuit.h(qubits[i])
        for j in range(i+1, len(qubits)):
            circuit.cp(2*np.pi/2**(j-i), qubits[j], qubits[i])
    for i in range(len(qubits)//2):
        circuit.swap(qubits[i], qubits[len(qubits)-1-i])
    return circuit

def aqft(qubits, max_rotations):
    circuit = QuantumCircuit(qubits)
    for i in range(len(qubits)):
        circuit.h(qubits[i])
        for j in range(i+1, min(i+max_rotations+1, len(qubits))):
            circuit.cp(2*np.pi/2**(j-i), qubits[j], qubits[i])
    for i in range(len(qubits)//2):
        circuit.swap(qubits[i], qubits[len(qubits)-1-i])
    return circuit

def phase_sum(qubits_A, qubits_B):
    circuit = QuantumCircuit(qubits_A, qubits_B)
    for i, q_A in enumerate(qubits_A):
        for j, q_B in enumerate(qubits_B):
            if j <= i:
                angle = np.pi/2**(i-j)
                circuit.cp(angle, q_B, q_A)
    return circuit

def draper_adder(qubits_A, qubits_B):
    circuit = QuantumCircuit(qubits_A, qubits_B)

    # QFT
    circuit.append(qft(qubits_A), qubits_A)

    # Suma en el dominio de la fase
    circuit.append(phase_sum(qubits_A, qubits_B), [qubits_A, qubits_B])

    # AQFT
    circuit.append(aqft(qubits_A, 2), qubits_A)

    # QFT inversa
    circuit.append(qft(qubits_A).inverse(), qubits_A)

    return circuit

qubits_A = QuantumRegister(4, 'q')
qubits_B = QuantumRegister(4, 'r')

circuit = QuantumCircuit(qubits_A, qubits_B, ClassicalRegister(8))

circuit.append(draper_adder(qubits_A, qubits_B), [qubits_A, qubits_B])

circuit.measure(qubits_A, range(4))
circuit.measure(qubits_B, range(4, 8))

# Seleccionamos el backend donde queremos ejecutar el circuito
backend = provider.backend.ibmq_lima

# Ejecutamos el circuito en el backend
job = execute(circuit, backend=backend, shots=8192)

# Obtenemos los resultados de la ejecución
result = job.result()

# Imprimimos los resultados
print(result.get_counts())


# No he sido capaz de solucionar el error que salta al llamar la funcion
# phase_sum, he intentado pasarlo como un solo qubit pero tampoco ha servido

ibmqfactory.load_account:WARNING:2023-03-22 12:17:39,352: Credentials are already in use. The existing account in the session will be replaced.


Traceback (most recent call last):
  Cell In[3], line 61
    circuit.append(draper_adder(qubits_A, qubits_B), [qubits_A, qubits_B])
  Cell In[3], line 46 in draper_adder
    circuit.append(phase_sum(qubits_A, qubits_B), [qubits_A, qubits_B])
  File /opt/conda/lib/python3.10/site-packages/qiskit/circuit/quantumcircuit.py:1264 in append
    for qarg, carg in operation.broadcast_arguments(expanded_qargs, expanded_cargs):
  File /opt/conda/lib/python3.10/site-packages/qiskit/circuit/instruction.py:481 in broadcast_arguments
    raise CircuitError(
CircuitError: 'The amount of qubit arguments 2 does not match the instruction expectation (8).'

Use %tb to get the full traceback.
